In [ ]:
# !pip install rdflib pydantic jinja2 networkx lxml
#2.1 Setup
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional
from pathlib import Path
import subprocess, json, time, uuid
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD
from jinja2 import Template

#2.2 Konfiguration (TwinCAT/OPC UA, KG, Pfade)
OPCUA_ENDPOINT = "opc.tcp://192.168.1.10:4840"   # dein Setup
KG_TTL_PATH = Path("kg/pfmea_msr.ttl")
OUT_DIR = Path("out"); OUT_DIR.mkdir(parents=True, exist_ok=True)

# Guardrail/HITL-Schalter
AGENT_ENABLED = True    # kann dein MSRGuard setzen/lesen
MAX_REACT_STEPS = 6



In [ ]:
#Event-Intake und ReAct-„Blackboard“-State
@dataclass
class UnknownFMEvent:
    fm_id: str
    context: Dict[str, Any]    # Sensorwerte, letzte Aktionen etc.

@dataclass
class AgentState:
    event: UnknownFMEvent
    retrieved: Dict[str, Any] = field(default_factory=dict)
    plan: List[Dict[str, Any]] = field(default_factory=list)
    artifacts: Dict[str, Any] = field(default_factory=dict)  # ST-Code, reports
    verdict: Optional[str] = None


In [ ]:
#Retrieve: KG-Suche (GraphRAG-Stil)
def kg_query_similar_failures(event: UnknownFMEvent) -> Dict[str, Any]:
    g = Graph()
    if KG_TTL_PATH.exists(): g.parse(KG_TTL_PATH)
    CASK = Namespace("http://caskade.org/ontology#")     # CaSkMan/CaSk Namensraum (Bezeichner exemplarisch)
    PFMEA = Namespace("http://pfmea-msr.org/ontology#")

    # sehr einfache Heuristik/Beispiel – später Text2SPARQL o. Ä.
    # Hole verwandte FailureModes, empfohlene Diagnoseschritte, existierende Reaktionen
    q = """
    PREFIX pf: <http://pfmea-msr.org/ontology#>
    SELECT ?knownFM ?diag ?reaction WHERE {
      ?knownFM a pf:FailureMode .
      OPTIONAL { ?knownFM pf:hasDiagnosticStep ?diag . }
      OPTIONAL { ?knownFM pf:hasReaction ?reaction . }
    } LIMIT 25
    """
    rows = list(g.query(q))
    return {"candidates": [{"fm": str(r.knownFM), "diag": str(r.diag) if r.diag else None,
                            "reaction": str(r.reaction) if r.reaction else None} for r in rows]}


In [ ]:
#Plan: minimaler Planner (JSON-Plan, deterministisch + ReAct-Loop)
def propose_plan(state: AgentState) -> List[Dict[str, Any]]:
    # deterministische Basiskette; später LLM-Planner (ReAct) einhängen
    plan = [
        {"step": "read_signals", "desc": "OPC UA read key sensors", "tool": "opcua.read", "args": {"nodes": ["ns=4;i=..."]}},
        {"step": "hypothesis", "desc": "derive likely cause using KG context", "tool": "kg.reason", "args": {}},
        {"step": "generate_st", "desc": "synthesize diagnostic ST routine", "tool": "codegen.st", "args": {"probe": "sensor_drift_check"}},
        {"step": "verify_st", "desc": "run formal/semiformal checks", "tool": "verify.st", "args": {}},
        {"step": "export_skill", "desc": "wrap as Skill + register in KG", "tool": "plc2skill.export", "args": {}},
        {"step": "validate_pfmea", "desc": "PFMEA-MSR policy check", "tool": "pfmea.validate", "args": {}},
    ]
    return plan


In [ ]:
#Code-Synthese: ST aus Template (LLM-unterstützt oder heuristisch)
ST_TEMPLATE = Template("""
PROGRAM {{ name }}
VAR
    {{ var_decl }}
END_VAR

(* Diagnostic probe: {{ probe_desc }} *)
{{ body }}
END_PROGRAM
""".strip())

def synthesize_st(probe_name: str, inputs: Dict[str, Any]) -> str:
    var_decl = "bStart : BOOL; xFault : BOOL;"
    body = "IF bStart THEN\n    (* probe logic *)\n    xFault := FALSE; (* TODO real check *)\nEND_IF;"
    st = ST_TEMPLATE.render(name=f"Diag_{probe_name}", var_decl=var_decl, body=body, probe_desc=probe_name)
    out = OUT_DIR / f"Diag_{probe_name}.st"
    out.write_text(st, encoding="utf-8")
    return str(out)


In [ ]:
#2.7 Verifikation (2 Wege)

#A. STbmc (postechsv/plc-release) – bounded model checking Script (Maude/Yices2).
#B. PLCverif – formale Verifikation für PLC-Programme. (Du kannst hier mit Subprozessen starten und später vertiefen.)

def verify_st_with_stbmc(st_path: str) -> Dict[str, Any]:
    # Platzhalter: rufe externes Tool auf, parse Ergebnisdatei/Exitcode
    try:
        # subprocess.run(["stbmc", st_path, "--k=50"], check=True)
        return {"ok": True, "tool": "stbmc", "details": "placeholder ok"}
    except Exception as e:
        return {"ok": False, "tool": "stbmc", "details": str(e)}

def verify_st_with_plcverif(st_path: str) -> Dict[str, Any]:
    # später echte PLCverif-Integration / nuXmv-Übersetzung
    return {"ok": True, "tool": "PLCverif", "details": "placeholder ok"}


In [ ]:
#2.8 Export als Skill + KG-Persistenz (CaSkMan/PLC2Skill-Stil)
def export_skill_and_register(st_path: str, event: UnknownFMEvent) -> Dict[str, Any]:
    g = Graph()
    if KG_TTL_PATH.exists(): g.parse(KG_TTL_PATH)
    CASK = Namespace("http://caskade.org/ontology#")
    PFMEA = Namespace("http://pfmea-msr.org/ontology#")
    EX = Namespace("http://example.org/exh#")
    g.bind("cask", CASK); g.bind("pf", PFMEA); g.bind("ex", EX)

    skill_id = f"Skill_{uuid.uuid4().hex[:8]}"
    skill_uri = EX[skill_id]
    g.add((skill_uri, RDF.type, CASK.Skill))
    g.add((skill_uri, RDFS.label, Literal(f"AutoDiag {event.fm_id}", datatype=XSD.string)))
    g.add((skill_uri, CASK.implementsCapability, PFMEA.DiagnosticCapability))
    g.add((skill_uri, CASK.artifactPath, Literal(st_path, datatype=XSD.string)))
    # Link: this skill addresses the unknown failure mode
    g.add((URIRef(event.fm_id), PFMEA.hasReaction, skill_uri))

    g.serialize(destination=str(KG_TTL_PATH), format="turtle")
    return {"skill_uri": str(skill_uri), "kg_path": str(KG_TTL_PATH)}

In [ ]:
#2.9 PFMEA-MSR-Validierung (Policy-Check/Guardrails)
def validate_against_pfmea(skill_uri: str) -> Dict[str, Any]:
    # placeholder: prüfe simple Policy (z.B. kein Schreibbefehl ohne Sperrbedingung)
    # später: Regeln aus PFMEA-MSR als SHACL/SPIN/Policy-Engine
    ok = True
    reasons = []
    return {"ok": ok, "reasons": reasons}

In [ ]:
#ReAct-Ausführung + Start/Stop
def run_unknown_failure_pipeline(event: UnknownFMEvent) -> AgentState:
    if not AGENT_ENABLED:
        raise RuntimeError("Agent disabled by HITL/Guardrail switch.")
    state = AgentState(event=event)

    # Thought: Retrieve
    state.retrieved = kg_query_similar_failures(event)

    # Plan
    state.plan = propose_plan(state)

    # Act/Observe loop (vereinfacht)
    for i, step in enumerate(state.plan, start=1):
        if not AGENT_ENABLED:
            state.verdict = "stopped_by_operator"; break
        if step["step"] == "generate_st":
            st_path = synthesize_st(step["args"]["probe"], {})
            state.artifacts["st_path"] = st_path
        elif step["step"] == "verify_st":
            r1 = verify_st_with_stbmc(state.artifacts["st_path"])
            r2 = verify_st_with_plcverif(state.artifacts["st_path"])
            state.artifacts["verify"] = {"stbmc": r1, "plcverif": r2}
            if not (r1["ok"] and r2["ok"]):
                state.verdict = "verification_failed"; break
        elif step["step"] == "export_skill":
            reg = export_skill_and_register(state.artifacts["st_path"], event)
            state.artifacts["skill"] = reg
        elif step["step"] == "validate_pfmea":
            v = validate_against_pfmea(state.artifacts["skill"]["skill_uri"])
            state.artifacts["pfmea"] = v
            state.verdict = "ok" if v["ok"] else "pfmea_blocked"
        # ... opcua.read / kg.reason würdest du hier ebenfalls einhängen
    return state

#Mini-Beispiel: Pipeline fahren
event = UnknownFMEvent(
    fm_id="http://pfmea-msr.org/fm/unknown_123",
    context={"symptoms": ["x sensor spikes", "conveyor stall"], "ts": time.time()}
)
state = run_unknown_failure_pipeline(event)
state.verdict, state.artifacts.get("skill")